# Multi objective optimization post-processing analysis

In [ ]:
#import WarmupFUSE
using FUSE
using Plots

In [ ]:
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_noonetime"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_slvv"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SMSEMOA"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SPEA2"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm_expressions_jfix_neo_SPEA2_HCD"
path= "optimization_run_LTSorHTS"

In [ ]:
IMAS.update_ExtractFunctionsLibrary!()
# use this to regenerate CSV cache file
dirs = filter(isdir,sort(readdir(path; join=true)))
println(length(dirs))
dirs = filter(x->!isfile(joinpath(x,"error.txt")) && isfile(joinpath(x,"dd.json")),dirs)#[1:100]
println(length(dirs))
outputs=FUSE.extract(dirs, filter_invalid=:cols);

# cache extrated information to CSV file
import DelimitedFiles
DelimitedFiles.writedlm(joinpath(path,"extract.csv"), Iterators.flatten(([names(outputs)], eachrow(outputs))), ',')

In [ ]:
# error analysis
dirs = filter(isdir,sort(readdir(path; join=true)))
errors=FUSE.categorize_errors(dirs;do_plot=true,show_first_line=true)
for (k,error) in enumerate(errors[:other])
    if k<100
        println(error)
    end
end

In [ ]:
# use this to load CSV cache file
outputs=FUSE.DataFrames.DataFrame(FUSE.CSV.File(joinpath(path,"extract.csv")));
println(length(outputs[:,1]))

In [ ]:
# this is just to list the fields that can be queried
IMAS.ExtractFunctionsLibrary

In [ ]:
N=length(outputs[:,"Pelectric_net"])
X=LinRange(1,N,50)

histogram2d(outputs[:,"Pelectric_net"],bins=(X,LinRange(0,500,50)),ylabel="Pelectric net", xlabel="Generation")
#scatter!(outputs[:,"Pelectric_net"],ylim=(-500,500))
display(hline!([200.0], ls=:dash, label=""))

histogram2d(outputs[:,"ip_ohm"],bins=(X,LinRange(-5,5,50)),ylabel="Ip ohmic", xlabel="Generation")
#scatter!(outputs[:,"ip_ohm"],ylim=(-50,50))
display(hline!([0.0], ls=:dash, label=""))

In [ ]:
import Statistics
for name in names(outputs)
    if !(typeof(outputs[1,name]) <: Number) || all(isnan.(outputs[:,name]))
        continue
    end
    y=outputs[:,name]
    y_nonan=y[@. !isnan.(y)]
    m=Statistics.median(y_nonan)
    σ=Statistics.median(Statistics.median(abs.(y_nonan.-m)))*10
    if σ != 0.0
        display(histogram2d(y,bins=(X,LinRange(max(m-σ,minimum(y_nonan)),min(m+σ,maximum(y_nonan)),100)),ylabel=name))
    end
end

In [ ]:
# x axis
xname="capital_cost"; x=outputs[:,xname]
#xname="βpol"; x=outputs[:,xname]
#xname="Pec [MW]"; x=outputs[:,"Pec"]

# y axis
yname="βn"; y=outputs[:,yname]
#yname="ip_bs/ip_bs_aux_ohm"; y=outputs[:,"ip_bs"]./outputs[:,"ip_bs_aux_ohm"];
#yname="Pec=R0*ne*ip_aux(5+zeff)/(Te*0.09)"; y= @. outputs[:,"<ne>"]/1E20*outputs[:,"R0"]*outputs[:,"ip_aux"]*(5.0+outputs[:,"<zeff>"])/(0.09*outputs[:,"<Te>"])

# z axis
#zname="R0"; z=outputs[:,zname]

# color
#cname="generation"; c=1:length(outputs[:,xname]);
#cname="Pelectric_net"; c=outputs[:,cname]
#cname="Ip aux [MA]"; c=outputs[:,"ip_aux"];
#cname="<zeff>"; c=outputs[:,"<zeff>"];
#cname="R0 [m]"; c=outputs[:,"R0"];
cname="B0 [T]"; c=outputs[:,"B0"];
#cname="ip_ohm"; c=outputs[:,cname]
#cname="ip"; c=outputs[:,cname]
#cname="ip_bs_aux_ohm/ip"; c=outputs[:,"ip_bs_aux_ohm"]./outputs[:,"ip"];
#cname="ip_bs_aux_ohm/ip"; c=outputs[:,"ip_bs_aux_ohm"]./outputs[:,"ip"];
#cname="Ip aux [MA]"; c=outputs[:,"ip_aux"]

clim=(-Inf,Inf)


# selection
sname0="Pelectric_net"; s0=outputs[:,sname0]
sname1="ip_ohm"; s1=outputs[:,sname1]
sname2="B0"; s2=outputs[:,sname2]

# subselection criterion
Δf=1.0
Δohm=1.0
n=length(x)
index=1:n
index=findall(  abs.(s0.-200).<Δf .&& (abs.(s1).<Δohm) .&& (index.> 000))
#index=findall( abs.(s0.-200).<Δf .&& (index.>0000))
#index=findall( abs.(s0.-200).<Δf)
#index=findall( (index.>n-1000))
annot=map(x->(x, :center, 3, "courier"), index)

# plotting
println("$(length(index)) points")
scatter(x[index], y[index], marker_z=c[index] ,xlabel=xname, ylabel=yname, colorbar_title=cname, marker=:circle,
    markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5,
#    series_annotations=annot,
#    aspect_ratio=:equal,
    xlim=(0,Inf), ylim=(0,Inf))
#    xlim=(0,10), ylim=(0,4.5))
#    xlim=(0,10), ylim=(0,5))

#Plots.abline!(1, 0, line=:dash, label="")

#plot!(x[index], 0.67.*outputs[:,"βpol"])
#plot!(x[index], 0.75.*outputs[:,"βpol"])

In [ ]:
n=9053
dir=outputs[n,"dir"]
#n=length(dirs)
println(dir)
dd,ini,act=FUSE.load(dir)
FUSE.ActorCosting(dd,act;model=:ARIES)
FUSE.digest(dd)